# * Sales Data

## Parameter

In [26]:
import configparser
import datetime as dt
import pandas as pd
import numpy as np
import xlrd
import oracledb
import re
import FN_Monitoring as fn

config = configparser.ConfigParser()
config.read('../../my_config.ini')
config.sections()

pd.set_option('future.no_silent_downcasting', True)

TDMDBPR_user = config['TDMDBPR']['username']
TDMDBPR_pwd = config['TDMDBPR']['password']
TDMDBPR_db = config['TDMDBPR']['db']
TDMDBPR_host = config['TDMDBPR']['host']
TDMDBPR_port = config['TDMDBPR']['port']

AKPIPRD_user = config['AKPIPRD']['username']
AKPIPRD_pwd = config['AKPIPRD']['password']
AKPIPRD_db = config['AKPIPRD']['db']
AKPIPRD_host = config['AKPIPRD']['host']
AKPIPRD_port = config['AKPIPRD']['port']

curr_dt = dt.datetime.now().date()
str_curr_dt = curr_dt.strftime('%Y%m%d')

## Actual : Monitoring
->  DIM_CORP_KPI

In [27]:
''' Execute Summary Data '''


# Input parameter
curr_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
v_start_date = 20240101
print(f'\nParameter input...\n')
print(f'   -> v_start_date: {v_start_date}')


# Connect : TDMDBPR
src_dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
src_conn = oracledb.connect(src_dsn)
print(f'\n{TDMDBPR_db} : Connected')
src_cur = src_conn.cursor()
query = (f"""
    SELECT /*+PARALLEL(8)*/
        SUBSTR(TM_KEY_DAY,1,6) AS TM_KEY_MTH, PRODUCT, COMP_CD, METRIC_CD, METRIC_NAME
        , MAX(LOAD_DATE) LOAD_DATE, MIN(TM_KEY_DAY) AS START_DAY, MAX(TM_KEY_DAY) AS END_DAY
        , SUM(CASE WHEN AREA_TYPE = 'C' THEN METRIC_VALUE END) C
        , SUM(CASE WHEN AREA_TYPE = 'P' THEN METRIC_VALUE END) P
        , SUM(CASE WHEN AREA_TYPE = 'G' THEN METRIC_VALUE END) G
        , SUM(CASE WHEN AREA_TYPE = 'H' THEN METRIC_VALUE END) H
        , SUM(CASE WHEN AREA_TYPE = 'HH' THEN METRIC_VALUE END) HH
        , SUM(CASE WHEN AREA_TYPE = 'CCAA' THEN METRIC_VALUE END) CCAA
        , SUM(CASE WHEN AREA_TYPE = 'CCAATT' THEN METRIC_VALUE END) CCAATT
        , COUNT(DISTINCT AREA_TYPE) CNT_AREA_TYPE, COUNT(DISTINCT AREA_CD) CNT_AREA_CD, COUNT(1) ROW_CNT
    FROM CDSAPPO.DIM_CORP_KPI
    WHERE REGEXP_LIKE(METRIC_CD, 'CS$|CG$') -->> 2025 KPIs
    AND EXISTS (
        SELECT 1 -->> AREA_CD IN (P, G, H, R, HH, CCAA, CCAATT)
        FROM (
            SELECT DISTINCT ORGID_P AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT ORGID_G AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT ORGID_R AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT ORGID_H AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT ORGID_HH AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT SUBSTR(CCAATT,1,4) AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT CCAATT AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
        ) TMP
        WHERE TMP_CD = AREA_CD )
    AND TM_KEY_DAY >= {v_start_date}
    GROUP BY SUBSTR(TM_KEY_DAY,1,6), PRODUCT, COMP_CD, METRIC_CD, METRIC_NAME
""")


try:
    execute_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
    print(f'\n   -> Execute query... {execute_datetime}')

    # Get : Summary DataFrame
    src_cur.execute(query)
    rows = src_cur.fetchall()
    chk_src_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in src_cur.description])
    print(f'\n   -> Summary DataFrame : {chk_src_df.shape[0]} rows, {chk_src_df.shape[1]} columns')

    src_cur.close()


except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    src_conn.close()
    print(f'\n{TDMDBPR_db} : Disconnected')


Parameter input...

   -> v_start_date: 20240101

TDMDBPR : Connected

   -> Execute query... 2025-04-03, 15:47:13

   -> Summary DataFrame : 1112 rows, 18 columns

TDMDBPR : Disconnected


In [28]:
def my_metric_group(v_name):

    # Get : Parameter
    name = v_name
    result = ''
    
    if re.search('Gross Adds', name): result = 'Gross Adds'
    elif re.search('Inflow M1', name): result = 'Inflow M1'
    elif re.search('Inflow M2', name): result = 'Inflow M2'
    elif re.search('No of AP 1D', name): result = 'No of AP 1D'
    elif re.search('Amt of AP 1D', name): result = 'Amt of AP 1D'
    elif re.search('No of AP In Month', name): result = 'No of AP In Month'
    elif re.search('Amt of AP In Month', name): result = 'Amt of AP In Month'
    # elif re.search('CConnected : onsumer', name): result = 'Gross Adds - Con - Consumer'
    # elif re.search('Connected : DataService', name): result = 'Gross Adds - Con - DataService'
    # elif re.search('Registration : Consumer', name): result = 'Gross Adds - Reg - Consumer'
    # elif re.search('Registration : DataService', name): result = 'Gross Adds - Reg - DataService'
    else: result = 'Unknown' 
    
    return result

In [29]:
''' Add Columns '''

chk_src_df['CHANNEL_GRP'] = np.where(chk_src_df['METRIC_CD'].str.contains('CS$'), 'ALL', 'GEO')
chk_src_df['LOCATION_GRP'] = np.where(chk_src_df['METRIC_NAME'].str.contains('Location'), 'Customer', 'Sales')
chk_src_df['METRIC_GRP'] = chk_src_df.apply(lambda x: my_metric_group(str(x['METRIC_NAME'])), axis=1)
# tmp_df['MAIN_FLAG'] = np.where((tmp_df['METRIC_CD'].str.contains('[0-9]C$|[0-9]H$|[0-9]MCOM$|[0-9]CUS$')) | (tmp_df['METRIC_NAME'].str.contains('New Revenue|Existing Revenue')), 'N', 'Y')
# chk_src_df.tail()

# chk_src_df.loc[chk_src_df['PRODUCT']=='TOL'].tail()

### KPIs Summary

In [30]:
''' PREPAID Summary : 14 rows, 24 KPIs '''

curr_mth = chk_src_df['TM_KEY_MTH'].max()
product = 'PREPAID'
# channel_grp = 'GEO'

prepaid_df = chk_src_df.copy()
prepaid_df = prepaid_df.loc[prepaid_df['TM_KEY_MTH']==curr_mth]
prepaid_df = prepaid_df.loc[prepaid_df['PRODUCT']==product]
# prepaid_df = prepaid_df.loc[prepaid_df['CHANNEL_GRP']==channel_grp]

prepaid_df['ALL'] = prepaid_df['P']
prepaid_df['TRUE'] = np.where(prepaid_df['COMP_CD']=='TRUE', prepaid_df['P'], 0)
prepaid_df['DTAC'] = np.where(prepaid_df['COMP_CD']=='DTAC', prepaid_df['P'], 0)
prepaid_df['TMP_NAME'] = prepaid_df['METRIC_NAME'].replace(r' : DTAC| : TMH', '', regex=True)

prepaid_df = prepaid_df.groupby(['PRODUCT', 'METRIC_GRP', 'TMP_NAME']).agg({'LOAD_DATE':'max', 'START_DAY':'min', 'END_DAY':'max', 'ALL':'sum', 'TRUE':'sum', 'DTAC':'sum'})
prepaid_df = prepaid_df.fillna(0).sort_values(by=['PRODUCT', 'METRIC_GRP', 'TMP_NAME']).reset_index()

mod_col_list = prepaid_df.iloc[:, 6:9].columns.tolist()
for col in mod_col_list:
    prepaid_df[col] = prepaid_df[col].apply(lambda x: format(x, ',.0f'))

prepaid_df

,PRODUCT,METRIC_GRP,TMP_NAME,LOAD_DATE,START_DAY,END_DAY,ALL,TRUE,DTAC
0,PREPAID,Amt of AP 1D,Prepaid Amt of AP 1D,2025-04-03 14:03:28,20250401,20250401,"5,766,569",0,"5,766,569"
1,PREPAID,Amt of AP 1D,Prepaid Amt of AP 1D - GEO Channel,2025-04-03 14:03:28,20250401,20250401,"5,549,841",0,"5,549,841"
2,PREPAID,Amt of AP In Month,Prepaid Amt of AP In Month,2025-04-03 14:03:28,20250401,20250401,"5,766,569",0,"5,766,569"
3,PREPAID,Amt of AP In Month,Prepaid Amt of AP In Month - GEO Channel,2025-04-03 14:03:28,20250401,20250401,"5,549,841",0,"5,549,841"
4,PREPAID,Gross Adds,Prepaid Gross Adds,2025-04-03 14:03:28,20250401,20250401,"30,296",0,"30,296"
5,PREPAID,Gross Adds,Prepaid Gross Adds - GEO Channel,2025-04-03 14:03:28,20250401,20250401,"29,327",0,"29,327"
6,PREPAID,Inflow M1,Prepaid Inflow M1,2025-04-03 14:03:28,20250401,20250401,"5,836,674",0,"5,836,674"
7,PREPAID,Inflow M1,Prepaid Inflow M1 - GEO Channel,2025-04-03 14:03:28,20250401,20250401,"5,623,271",0,"5,623,271"
8,PREPAID,Inflow M2,Prepaid Inflow M2,2025-04-03 14:03:28,20250401,20250401,"2,848,602",0,"2,848,602"
9,PREPAID,Inflow M2,Prepaid Inflow M2 - GEO Channel,2025-04-03 14:03:28,20250401,20250401,"2,789,629",0,"2,789,629"


In [31]:
''' POSTPAID Summary : 8 rows, 16 KPIs '''

curr_mth = chk_src_df['TM_KEY_MTH'].max()
product = 'POSTPAID'
# channel_grp = 'GEO'

postpaid_df = chk_src_df.copy()
postpaid_df = postpaid_df.loc[postpaid_df['TM_KEY_MTH']==curr_mth]
postpaid_df = postpaid_df.loc[postpaid_df['PRODUCT']==product]
# postpaid_df = postpaid_df.loc[postpaid_df['CHANNEL_GRP']==channel_grp]

postpaid_df['ALL'] = postpaid_df['P']
postpaid_df['TRUE'] = np.where(postpaid_df['COMP_CD']=='TRUE', postpaid_df['P'], 0)
postpaid_df['DTAC'] = np.where(postpaid_df['COMP_CD']=='DTAC', postpaid_df['P'], 0)
postpaid_df['TMP_NAME'] = postpaid_df['METRIC_NAME'].replace(r' : DTAC| : TMH', '', regex=True)

postpaid_df = postpaid_df.groupby(['PRODUCT', 'METRIC_GRP', 'TMP_NAME']).agg({'LOAD_DATE':'max', 'START_DAY':'min', 'END_DAY':'max', 'ALL':'sum', 'TRUE':'sum', 'DTAC':'sum'})
postpaid_df = postpaid_df.fillna(0).sort_values(by=['PRODUCT', 'METRIC_GRP', 'TMP_NAME']).reset_index()

mod_col_list = postpaid_df.iloc[:, 6:9].columns.tolist()
for col in mod_col_list:
    postpaid_df[col] = postpaid_df[col].apply(lambda x: format(x, ',.0f'))

postpaid_df

,PRODUCT,METRIC_GRP,TMP_NAME,LOAD_DATE,START_DAY,END_DAY,ALL,TRUE,DTAC
0,POSTPAID,Gross Adds,Postpaid Gross Adds B2B,2025-04-03 14:15:44,20250401,20250402,"1,788","1,471",317
1,POSTPAID,Gross Adds,Postpaid Gross Adds B2B - GEO Channel,2025-04-03 14:15:44,20250401,20250402,674,612,62
2,POSTPAID,Gross Adds,Postpaid Gross Adds B2C,2025-04-03 14:15:44,20250401,20250402,"14,443","13,171","1,272"
3,POSTPAID,Gross Adds,Postpaid Gross Adds B2C - GEO Channel,2025-04-03 14:15:44,20250401,20250402,"7,808","7,267",541
4,POSTPAID,Inflow M1,Postpaid Inflow M1 B2B,2025-04-03 14:15:44,20250401,20250402,"2,536,525","961,529","1,574,996"
5,POSTPAID,Inflow M1,Postpaid Inflow M1 B2B - GEO Channel,2025-04-03 14:15:44,20250401,20250402,"593,807","555,418","38,389"
6,POSTPAID,Inflow M1,Postpaid Inflow M1 B2C,2025-04-03 14:15:44,20250401,20250402,"6,099,626","5,527,142","572,484"
7,POSTPAID,Inflow M1,Postpaid Inflow M1 B2C - GEO Channel,2025-04-03 14:15:44,20250401,20250402,"3,701,639","3,396,061","305,578"


In [32]:
''' TOL Summary : 20 KPIs'''

curr_mth = chk_src_df['TM_KEY_MTH'].max()
product = 'TOL'
# channel_grp = 'GEO'

tol_df = chk_src_df.copy()
tol_df = tol_df.loc[tol_df['TM_KEY_MTH']==curr_mth]
tol_df = tol_df.loc[tol_df['PRODUCT']==product]
# tol_df = tol_df.loc[tol_df['CHANNEL_GRP']==channel_grp]

tol_df = tol_df.groupby(['PRODUCT', 'METRIC_GRP', 'METRIC_NAME']).agg({'LOAD_DATE':'max', 'START_DAY':'min', 'END_DAY':'max', 'P':'sum'})
tol_df = tol_df.fillna(0).sort_values(by=['PRODUCT', 'METRIC_GRP', 'METRIC_NAME']).reset_index()

mod_col_list = tol_df.iloc[:, 6:7].columns.tolist()
for col in mod_col_list:
    tol_df[col] = tol_df[col].apply(lambda x: format(x, ',.0f'))

tol_df

,PRODUCT,METRIC_GRP,METRIC_NAME,LOAD_DATE,START_DAY,END_DAY,P
0,TOL,Gross Adds,TOL Gross Adds Connected : Consumer,2025-04-03 14:02:38,20250401,20250401,"1,787"
1,TOL,Gross Adds,TOL Gross Adds Connected : Consumer (Install L...,2025-04-03 14:02:38,20250401,20250401,"1,787"
2,TOL,Gross Adds,TOL Gross Adds Connected : Consumer - GEO Channel,2025-04-03 14:02:38,20250401,20250401,"1,486"
3,TOL,Gross Adds,TOL Gross Adds Connected : Consumer - GEO Chan...,2025-04-03 14:02:38,20250401,20250401,"1,486"
4,TOL,Gross Adds,TOL Gross Adds Connected : DataService,2025-04-03 14:02:38,20250401,20250401,93
5,TOL,Gross Adds,TOL Gross Adds Connected : DataService (Instal...,2025-04-03 14:02:38,20250401,20250401,93
6,TOL,Gross Adds,TOL Gross Adds Connected : DataService - GEO C...,2025-04-03 14:02:38,20250401,20250401,7
7,TOL,Gross Adds,TOL Gross Adds Connected : DataService - GEO C...,2025-04-03 14:02:38,20250401,20250401,7
8,TOL,Gross Adds,TOL Gross Adds Registration : Consumer,2025-04-03 14:02:38,20250401,20250401,"2,386"
9,TOL,Gross Adds,TOL Gross Adds Registration : Consumer - GEO C...,2025-04-03 14:02:38,20250401,20250401,"1,848"


In [33]:
''' TVS Summary : 8 KPIs '''

curr_mth = chk_src_df['TM_KEY_MTH'].max()
product = 'TVS'
# channel_grp = 'GEO'

tvs_df = chk_src_df.copy()
tvs_df = tvs_df.loc[tvs_df['TM_KEY_MTH']==curr_mth]
tvs_df = tvs_df.loc[tvs_df['PRODUCT']==product]
# tvs_df = tvs_df.loc[tvs_df['CHANNEL_GRP']==channel_grp]

tvs_df = tvs_df.groupby(['PRODUCT', 'METRIC_GRP', 'METRIC_NAME']).agg({'LOAD_DATE':'max', 'START_DAY':'min', 'END_DAY':'max', 'P':'sum'})
tvs_df = tvs_df.fillna(0).sort_values(by=['PRODUCT', 'METRIC_GRP', 'METRIC_NAME']).reset_index()

mod_col_list = tvs_df.iloc[:, 6:7].columns.tolist()
for col in mod_col_list:
    tvs_df[col] = tvs_df[col].apply(lambda x: format(x, ',.0f'))

tvs_df

,PRODUCT,METRIC_GRP,METRIC_NAME,LOAD_DATE,START_DAY,END_DAY,P
0,TVS,Gross Adds,TVS CMDU Gross Adds,2025-04-03 14:18:45,20250401,20250401,7
1,TVS,Gross Adds,TVS CMDU Gross Adds - GEO Channel,2025-04-03 14:18:45,20250401,20250401,3
2,TVS,Gross Adds,TVS Now Gross Adds,2025-04-03 14:18:45,20250401,20250401,29
3,TVS,Gross Adds,TVS Now Gross Adds - GEO Channel,2025-04-03 14:18:45,20250401,20250401,29
4,TVS,Inflow M1,TVS CMDU Inflow M1,2025-04-03 14:18:45,20250401,20250401,"56,507"
5,TVS,Inflow M1,TVS CMDU Inflow M1 - GEO Channel,2025-04-03 14:18:45,20250401,20250401,"22,498"
6,TVS,Inflow M1,TVS Now Inflow M1,2025-04-03 14:18:45,20250401,20250401,"9,268"
7,TVS,Inflow M1,TVS Now Inflow M1 - GEO Channel,2025-04-03 14:18:45,20250401,20250401,"9,268"


### Monitoring

In [34]:
''' Current Month Gross Adds '''

curr_mth = chk_src_df['TM_KEY_MTH'].max()
ga_metric_list = [
    'DB2S010100CS' #Postpaid Gross Adds B2C : DTAC
    , 'DB2S020100CS' #Postpaid Gross Adds B2B : DTAC
    , 'TB2S010100CS' #Postpaid Gross Adds B2C : TMH
    , 'TB2S020100CS' #Postpaid Gross Adds B2B : TMH
    , 'DB1S000101CS' #Prepaid Gross Adds : DTAC
    , 'TB1S000101CS' #Prepaid Gross Adds : TMH
    , 'TB3S000102CS' #TOL Gross Adds Connected : Consumer
    , 'TB3S000103CS' #TOL Gross Adds Connected : DataService
    , 'TB3S000104CS' #TOL Gross Adds Registration : Consumer
    , 'TB3S000105CS' #TOL Gross Adds Registration : DataService
    , 'TB4S001004CS' #TVS CMDU Gross Adds
    , 'TB4S001400CS' #TVS Now Gross Adds
    ]

src_ga_overall_df = chk_src_df.copy()
src_ga_overall_df = src_ga_overall_df.loc[src_ga_overall_df['TM_KEY_MTH']==curr_mth]
src_ga_overall_df = src_ga_overall_df.loc[src_ga_overall_df['METRIC_CD'].isin(ga_metric_list)]
src_ga_overall_df = src_ga_overall_df.groupby(['PRODUCT', 'METRIC_CD', 'METRIC_NAME']).agg({'LOAD_DATE':'max', 'START_DAY':'min', 'END_DAY':'max', 'C':'sum', 'P':'sum', 'G':'sum', 'H':'sum', 'HH':'sum', 'CCAA':'sum', 'CCAATT':'sum', 'CNT_AREA_TYPE':'max', 'CNT_AREA_CD':'max', 'ROW_CNT':'sum'})
src_ga_overall_df = src_ga_overall_df.fillna(0).sort_values(by=['PRODUCT', 'METRIC_CD']).reset_index()

mod_col_list = src_ga_overall_df.iloc[:, 6:13].columns.tolist()
for col in mod_col_list:
    src_ga_overall_df[col] = src_ga_overall_df[col].apply(lambda x: format(x, ',.0f'))

src_ga_overall_df

,PRODUCT,METRIC_CD,METRIC_NAME,LOAD_DATE,START_DAY,END_DAY,C,P,G,H,HH,CCAA,CCAATT,CNT_AREA_TYPE,CNT_AREA_CD,ROW_CNT
0,POSTPAID,DB2S010100CS,Postpaid Gross Adds B2C : DTAC,2025-04-03 14:15:44,20250401,20250401,0,"1,272","1,257","1,257","1,257","1,257","1,257",7,1198,1198
1,POSTPAID,DB2S020100CS,Postpaid Gross Adds B2B : DTAC,2025-04-03 14:15:44,20250401,20250401,0,317,102,102,102,102,102,7,154,154
2,POSTPAID,TB2S010100CS,Postpaid Gross Adds B2C : TMH,2025-04-03 14:15:44,20250401,20250402,0,"13,171","9,701","9,701","9,701","9,701","9,701",7,1069,1892
3,POSTPAID,TB2S020100CS,Postpaid Gross Adds B2B : TMH,2025-04-03 14:15:44,20250401,20250402,0,"1,471",613,613,613,613,613,7,293,386
4,PREPAID,DB1S000101CS,Prepaid Gross Adds : DTAC,2025-04-03 14:03:28,20250401,20250401,0,"30,296","25,090","25,090","28,106","28,472","28,276",7,3476,3476
5,TOL,TB3S000102CS,TOL Gross Adds Connected : Consumer,2025-04-03 14:02:38,20250401,20250401,0,"1,787","1,486","1,486","1,475","1,486","1,486",7,751,751
6,TOL,TB3S000103CS,TOL Gross Adds Connected : DataService,2025-04-03 14:02:38,20250401,20250401,0,93,7,7,7,7,7,7,21,21
7,TOL,TB3S000104CS,TOL Gross Adds Registration : Consumer,2025-04-03 14:02:38,20250401,20250401,0,"2,386","1,848","1,848","1,835","1,848","1,848",7,788,788
8,TOL,TB3S000105CS,TOL Gross Adds Registration : DataService,2025-04-03 14:02:38,20250401,20250401,0,101,1,1,1,1,1,7,7,7
9,TVS,TB4S001004CS,TVS CMDU Gross Adds,2025-04-03 14:18:45,20250401,20250401,0,7,7,7,7,7,7,7,27,27


In [35]:
''' Average Gross Adds per Month '''

avg_range_start = '202401'
avg_range_end = '202503'
ga_metric_list = [
    'DB2S010100CS' #Postpaid Gross Adds B2C : DTAC
    , 'DB2S020100CS' #Postpaid Gross Adds B2B : DTAC
    , 'TB2S010100CS' #Postpaid Gross Adds B2C : TMH
    , 'TB2S020100CS' #Postpaid Gross Adds B2B : TMH
    , 'DB1S000101CS' #Prepaid Gross Adds : DTAC
    , 'TB1S000101CS' #Prepaid Gross Adds : TMH
    , 'TB3S000102CS' #TOL Gross Adds Connected : Consumer
    , 'TB3S000103CS' #TOL Gross Adds Connected : DataService
    , 'TB3S000104CS' #TOL Gross Adds Registration : Consumer
    , 'TB3S000105CS' #TOL Gross Adds Registration : DataService
    , 'TB4S001004CS' #TVS CMDU Gross Adds
    , 'TB4S001400CS' #TVS Now Gross Adds
    ]

src_ga_overall_df = chk_src_df.copy()
src_ga_overall_df = src_ga_overall_df.loc[src_ga_overall_df['TM_KEY_MTH'].between(avg_range_start, avg_range_end)]
src_ga_overall_df = src_ga_overall_df.loc[src_ga_overall_df['METRIC_CD'].isin(ga_metric_list)]
src_ga_overall_df = src_ga_overall_df.groupby(['PRODUCT', 'METRIC_CD', 'METRIC_NAME']).agg({'LOAD_DATE':'max', 'START_DAY':'min', 'END_DAY':'max', 'C':'mean', 'P':'mean', 'G':'mean', 'H':'mean', 'HH':'mean', 'CCAA':'mean', 'CCAATT':'mean', 'CNT_AREA_TYPE':'max', 'CNT_AREA_CD':'max', 'ROW_CNT':'sum'})
src_ga_overall_df = src_ga_overall_df.fillna(0).sort_values(by=['PRODUCT', 'METRIC_CD']).reset_index()

mod_col_list = src_ga_overall_df.iloc[:, 6:13].columns.tolist()
for col in mod_col_list:
    src_ga_overall_df[col] = src_ga_overall_df[col].apply(lambda x: format(x, ',.0f'))

src_ga_overall_df

,PRODUCT,METRIC_CD,METRIC_NAME,LOAD_DATE,START_DAY,END_DAY,C,P,G,H,HH,CCAA,CCAATT,CNT_AREA_TYPE,CNT_AREA_CD,ROW_CNT
0,POSTPAID,DB2S010100CS,Postpaid Gross Adds B2C : DTAC,2025-04-03 14:15:44,20240101,20250331,0,"61,560","59,719","59,719","59,719","59,719","59,719",7,6204,697805
1,POSTPAID,DB2S020100CS,Postpaid Gross Adds B2B : DTAC,2025-04-03 14:15:44,20240101,20250331,0,"10,055","5,344","5,344","5,344","5,344","5,344",7,1528,80114
2,POSTPAID,TB2S010100CS,Postpaid Gross Adds B2C : TMH,2025-04-03 14:15:44,20240101,20250331,0,"152,587","103,167","103,167","103,167","103,167","103,167",7,2211,438949
3,POSTPAID,TB2S020100CS,Postpaid Gross Adds B2B : TMH,2025-04-03 14:15:44,20240101,20250331,0,"25,016","8,124","8,124","8,124","8,124","8,124",7,781,64761
4,PREPAID,DB1S000101CS,Prepaid Gross Adds : DTAC,2025-04-03 14:03:28,20240101,20250330,0,"1,344,807","1,285,585","1,285,585","1,281,479","1,299,408","1,291,516",7,7441,1680926
5,PREPAID,TB1S000101CS,Prepaid Gross Adds : TMH,2025-04-03 14:03:28,20240101,20250331,0,"1,138,164","940,454","940,454","918,384","940,454","940,390",7,5820,1789280
6,TOL,TB3S000102CS,TOL Gross Adds Connected : Consumer,2025-04-03 14:02:38,20240102,20250331,0,"46,948","33,562","33,562","34,737","35,069","35,055",7,1214,331274
7,TOL,TB3S000103CS,TOL Gross Adds Connected : DataService,2025-04-03 14:02:38,20240102,20250331,0,"1,584",34,34,35,35,35,7,99,2156
8,TOL,TB3S000104CS,TOL Gross Adds Registration : Consumer,2025-04-03 14:02:38,20240101,20250331,0,"64,322","45,578","45,578","47,202","47,631","47,608",7,1243,348927
9,TOL,TB3S000105CS,TOL Gross Adds Registration : DataService,2025-04-03 14:02:38,20240101,20250331,0,"3,353","1,033","1,033","1,025","1,043","1,043",7,620,25738


In [36]:
''' Current Month Inflow '''

curr_mth = chk_src_df['TM_KEY_MTH'].max()
inflow_metric_list = [
    'DB2R010500CS' #Postpaid Inflow M1 B2C : DTAC
    , 'DB2R020500CS' #Postpaid Inflow M1 B2B : DTAC
    , 'TB2R010500CS' #Postpaid Inflow M1 B2C : TMH
    , 'TB2R020500CS' #Postpaid Inflow M1 B2B : TMH
    , 'DB1R000900CS' #Prepaid Inflow M1 : DTAC
    , 'TB1R000900CS' #Prepaid Inflow M1 : TMH
    , 'TB3R000601CS' #TOL Inflow M1 Connected : Consumer
    , 'TB3R000602CS' #TOL Inflow M1 Connected : DataService
    , 'TB4R001004CS' #TVS CMDU Inflow M1
    , 'TB4R001700CS' #TVS Now Inflow M1
    , 'DB1R001000CS' #Prepaid Inflow M2 : DTAC
    , 'TB1R001000CS' #Prepaid Inflow M2 : TMH
    ]

src_inflow_overall_df = chk_src_df.copy()
src_inflow_overall_df = src_inflow_overall_df.loc[src_inflow_overall_df['TM_KEY_MTH']==curr_mth]
src_inflow_overall_df = src_inflow_overall_df.loc[src_inflow_overall_df['METRIC_CD'].isin(inflow_metric_list)]
src_inflow_overall_df = src_inflow_overall_df.groupby(['PRODUCT', 'METRIC_CD', 'METRIC_NAME']).agg({'LOAD_DATE':'max', 'START_DAY':'min', 'END_DAY':'max', 'C':'sum', 'P':'sum', 'G':'sum', 'H':'sum', 'HH':'sum', 'CCAA':'sum', 'CCAATT':'sum', 'CNT_AREA_TYPE':'max', 'CNT_AREA_CD':'max', 'ROW_CNT':'sum'})
src_inflow_overall_df = src_inflow_overall_df.fillna(0).sort_values(by=['PRODUCT', 'METRIC_CD']).reset_index()

mod_col_list = src_inflow_overall_df.iloc[:, 6:13].columns.tolist()
for col in mod_col_list:
    src_inflow_overall_df[col] = src_inflow_overall_df[col].apply(lambda x: format(x, ',.0f'))

src_inflow_overall_df

,PRODUCT,METRIC_CD,METRIC_NAME,LOAD_DATE,START_DAY,END_DAY,C,P,G,H,HH,CCAA,CCAATT,CNT_AREA_TYPE,CNT_AREA_CD,ROW_CNT
0,POSTPAID,DB2R010500CS,Postpaid Inflow M1 B2C : DTAC,2025-04-03 14:15:44,20250401,20250401,0,"572,484","564,198","564,198","564,198","564,198","564,198",7,1198,1198
1,POSTPAID,DB2R020500CS,Postpaid Inflow M1 B2B : DTAC,2025-04-03 14:15:44,20250401,20250401,0,"1,574,996","66,218","66,218","66,218","66,218","66,218",7,154,154
2,POSTPAID,TB2R010500CS,Postpaid Inflow M1 B2C : TMH,2025-04-03 14:15:44,20250401,20250402,0,"5,527,142","4,543,402","4,543,402","4,543,402","4,543,402","4,543,402",7,1069,1892
3,POSTPAID,TB2R020500CS,Postpaid Inflow M1 B2B : TMH,2025-04-03 14:15:44,20250401,20250402,0,"961,529","555,917","555,917","555,917","555,917","555,917",7,293,386
4,PREPAID,DB1R000900CS,Prepaid Inflow M1 : DTAC,2025-04-03 14:03:28,20250401,20250401,0,"5,836,674","5,025,027","5,025,027","5,519,223","5,595,888","5,558,751",7,3467,3467
5,PREPAID,DB1R001000CS,Prepaid Inflow M2 : DTAC,2025-04-03 14:03:28,20250401,20250401,0,"2,848,602","2,414,829","2,414,829","2,744,467","2,778,526","2,744,256",7,6032,6032
6,TOL,TB3R000601CS,TOL Inflow M1 Connected : Consumer,2025-04-03 14:02:38,20250401,20250401,0,"1,047,649","876,802","876,802","869,844","876,802","876,802",7,751,751
7,TOL,TB3R000602CS,TOL Inflow M1 Connected : DataService,2025-04-03 14:02:38,20250401,20250401,0,"23,953","7,796","7,796","7,796","7,796","7,796",7,21,21
8,TVS,TB4R001004CS,TVS CMDU Inflow M1,2025-04-03 14:18:45,20250401,20250401,0,"56,507","56,507","56,507","56,507","56,507","56,507",7,27,27
9,TVS,TB4R001700CS,TVS Now Inflow M1,2025-04-03 14:18:45,20250401,20250401,0,"9,268","9,268","9,268","9,268","9,268","9,268",7,101,101


In [37]:
''' Average Inflow per Month '''

avg_range_start = '202401'
avg_range_end = '202503'
inflow_metric_list = [
    'DB2R010500CS' #Postpaid Inflow M1 B2C : DTAC
    , 'DB2R020500CS' #Postpaid Inflow M1 B2B : DTAC
    , 'TB2R010500CS' #Postpaid Inflow M1 B2C : TMH
    , 'TB2R020500CS' #Postpaid Inflow M1 B2B : TMH
    , 'DB1R000900CS' #Prepaid Inflow M1 : DTAC
    , 'TB1R000900CS' #Prepaid Inflow M1 : TMH
    , 'TB3R000601CS' #TOL Inflow M1 Connected : Consumer
    , 'TB3R000602CS' #TOL Inflow M1 Connected : DataService
    , 'TB4R001004CS' #TVS CMDU Inflow M1
    , 'TB4R001700CS' #TVS Now Inflow M1
    , 'DB1R001000CS' #Prepaid Inflow M2 : DTAC
    , 'TB1R001000CS' #Prepaid Inflow M2 : TMH
    ]

src_inflow_overall_df = chk_src_df.copy()
src_inflow_overall_df = src_inflow_overall_df.loc[src_inflow_overall_df['TM_KEY_MTH'].between(avg_range_start, avg_range_end)]
src_inflow_overall_df = src_inflow_overall_df.loc[src_inflow_overall_df['METRIC_CD'].isin(inflow_metric_list)]
src_inflow_overall_df = src_inflow_overall_df.groupby(['PRODUCT', 'METRIC_CD', 'METRIC_NAME']).agg({'LOAD_DATE':'max', 'START_DAY':'min', 'END_DAY':'max', 'C':'sum', 'P':'sum', 'G':'sum', 'H':'sum', 'HH':'sum', 'CCAA':'sum', 'CCAATT':'sum', 'CNT_AREA_TYPE':'max', 'CNT_AREA_CD':'max', 'ROW_CNT':'sum'})
src_inflow_overall_df = src_inflow_overall_df.fillna(0).sort_values(by=['PRODUCT', 'METRIC_CD']).reset_index()

mod_col_list = src_inflow_overall_df.iloc[:, 6:13].columns.tolist()
for col in mod_col_list:
    src_inflow_overall_df[col] = src_inflow_overall_df[col].apply(lambda x: format(x, ',.0f'))

src_inflow_overall_df

,PRODUCT,METRIC_CD,METRIC_NAME,LOAD_DATE,START_DAY,END_DAY,C,P,G,H,HH,CCAA,CCAATT,CNT_AREA_TYPE,CNT_AREA_CD,ROW_CNT
0,POSTPAID,DB2R010500CS,Postpaid Inflow M1 B2C : DTAC,2025-04-03 14:15:44,20240101,20250331,0,"399,211,856","390,221,035","390,221,035","390,221,035","390,221,035","390,221,035",7,6204,697805
1,POSTPAID,DB2R020500CS,Postpaid Inflow M1 B2B : DTAC,2025-04-03 14:15:44,20240101,20250331,0,"146,147,330","27,548,995","27,548,995","27,548,995","27,548,995","27,548,995",7,1528,80114
2,POSTPAID,TB2R010500CS,Postpaid Inflow M1 B2C : TMH,2025-04-03 14:15:44,20240101,20250331,0,"963,880,802","742,681,740","742,681,740","742,681,740","742,681,740","742,681,740",7,2211,438949
3,POSTPAID,TB2R020500CS,Postpaid Inflow M1 B2B : TMH,2025-04-03 14:15:44,20240101,20250331,0,"84,636,549","36,643,396","36,643,396","36,643,396","36,643,396","36,643,396",7,781,64761
4,PREPAID,DB1R000900CS,Prepaid Inflow M1 : DTAC,2025-04-03 14:03:28,20240101,20250331,0,"3,795,698,843","3,689,485,262","3,689,485,262","3,675,061,056","3,733,676,852","3,708,885,577",7,7418,2535893
5,PREPAID,DB1R001000CS,Prepaid Inflow M2 : DTAC,2025-04-03 14:03:28,20240201,20250331,0,"1,083,317,243","1,050,794,381","1,050,794,381","1,056,961,196","1,069,475,778","1,057,293,456",7,7294,2747604
6,PREPAID,TB1R000900CS,Prepaid Inflow M1 : TMH,2025-04-03 14:03:28,20240101,20250331,0,"2,938,521,472","2,535,139,124","2,535,139,124","2,461,497,567","2,535,139,124","2,534,975,883",7,5793,2096677
7,PREPAID,TB1R001000CS,Prepaid Inflow M2 : TMH,2025-04-03 14:03:28,20240101,20250331,0,"1,060,137,134","1,049,549,983","1,049,549,983","1,030,456,436","1,049,549,983","1,049,435,343",7,5739,2269687
8,TOL,TB3R000601CS,TOL Inflow M1 Connected : Consumer,2025-04-03 14:02:38,20240102,20250331,0,"388,816,482","283,271,926","283,271,926","293,326,483","296,388,584","296,265,959",7,1214,331274
9,TOL,TB3R000602CS,TOL Inflow M1 Connected : DataService,2025-04-03 14:02:38,20240102,20250331,0,"29,097,110","1,591,398","1,591,398","1,627,784","1,630,883","1,630,883",7,99,2156


### All Metric

In [38]:
''' METRIC Summary '''

# src_t_sales_mth_df = chk_src_df.copy()
# src_t_sales_mth_df = src_t_sales_mth_df.groupby(['PRODUCT', 'METRIC_CD', 'METRIC_NAME']).agg({'LOAD_DATE':'max', 'START_DAY':'min', 'END_DAY':'max', 'P':'sum', 'CNT_AREA_TYPE':'max', 'CNT_AREA_CD':'max', 'ROW_CNT':'sum'})
# src_t_sales_mth_df = src_t_sales_mth_df.fillna(0).sort_values(by=['PRODUCT', 'METRIC_CD']).reset_index()

# mod_col_list = src_t_sales_mth_df.iloc[:, 6:7].columns.tolist()
# for col in mod_col_list:
#     src_t_sales_mth_df[col] = src_t_sales_mth_df[col].apply(lambda x: format(x, ',.0f'))

# print(f'\n{src_t_sales_mth_df.to_string(max_cols=10)}')

' METRIC Summary '

In [39]:
''' METRIC Current Month '''

# # v_tm_key_mth = '202502'
# curr_mth = chk_src_df['TM_KEY_MTH'].max()
# src_t_sales_mth_df_cols = ['PRODUCT', 'METRIC_CD', 'METRIC_NAME', 'LOAD_DATE', 'TM_KEY_MTH', 'START_DAY', 'END_DAY', 'C', 'P', 'G', 'H', 'HH', 'CCAA', 'CCAATT']
# print(f'\nParameter input...\n')
# print(f'   -> v_tm_key_mth: {curr_mth}')

# src_t_sales_mth_df = chk_src_df[src_t_sales_mth_df_cols].copy()
# src_t_sales_mth_df = src_t_sales_mth_df.loc[src_t_sales_mth_df['TM_KEY_MTH']==curr_mth]
# src_t_sales_mth_df = src_t_sales_mth_df.fillna(0).sort_values(by=['PRODUCT', 'METRIC_CD', 'TM_KEY_MTH']).reset_index(drop=True)

# mod_col_list = src_t_sales_mth_df.iloc[:, 7:14].columns.tolist()
# for col in mod_col_list:
#     src_t_sales_mth_df[col] = src_t_sales_mth_df[col].apply(lambda x: format(x, ',.0f'))

# print(f'\n{src_t_sales_mth_df.to_string(max_cols=17)}')

' METRIC Current Month '

## Actual : Transaction
->  DIM_CORP_KPI

In [40]:
''' Execute Daily by case '''


# Input parameter
curr_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
v_start_date = 20250101
print(f'\nParameter input...\n')
print(f'   -> v_start_date: {v_start_date}')


# Connect : TDMDBPR
src_dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
src_conn = oracledb.connect(src_dsn)
print(f'\n{TDMDBPR_db} : Connected')
src_cur = src_conn.cursor()
query = (f"""
    SELECT /*+PARALLEL(8)*/
        PRODUCT, METRIC_CD, METRIC_NAME, TM_KEY_DAY
        , MAX(LOAD_DATE) LOAD_DATE
        , SUM(CASE WHEN AREA_TYPE = 'C' THEN METRIC_VALUE END) C
        , SUM(CASE WHEN AREA_TYPE = 'P' THEN METRIC_VALUE END) P
        , SUM(CASE WHEN AREA_TYPE = 'G' THEN METRIC_VALUE END) G
        , SUM(CASE WHEN AREA_TYPE = 'H' THEN METRIC_VALUE END) H
        , SUM(CASE WHEN AREA_TYPE = 'HH' THEN METRIC_VALUE END) HH
        , SUM(CASE WHEN AREA_TYPE = 'CCAA' THEN METRIC_VALUE END) CCAA
        , SUM(CASE WHEN AREA_TYPE = 'CCAATT' THEN METRIC_VALUE END) CCAATT
    FROM CDSAPPO.DIM_CORP_KPI
    WHERE REGEXP_LIKE(METRIC_CD, 'CS$|CG$') -->> 2025 KPIs
    AND EXISTS (
        SELECT 1 -->> AREA_CD IN (P, G, H, R, HH, CCAA, CCAATT)
        FROM (
            SELECT DISTINCT ORGID_P AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT ORGID_G AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT ORGID_R AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT ORGID_H AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT ORGID_HH AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT SUBSTR(CCAATT,1,4) AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT CCAATT AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
        ) TMP
        WHERE TMP_CD = AREA_CD )
    AND TM_KEY_DAY >= {v_start_date}
    GROUP BY PRODUCT, METRIC_CD, METRIC_NAME, TM_KEY_DAY
""")


try:
    execute_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
    print(f'\n   -> Execute query... {execute_datetime}')

    # Get : Summary DataFrame
    src_cur.execute(query)
    rows = src_cur.fetchall()
    chk_src_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in src_cur.description])
    print(f'\n   -> Daily DataFrame : {chk_src_df.shape[0]} rows, {chk_src_df.shape[1]} columns')

    src_cur.close()


except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    src_conn.close()
    print(f'\n{TDMDBPR_db} : Disconnected')


Parameter input...

   -> v_start_date: 20250101

TDMDBPR : Connected

   -> Execute query... 2025-04-03, 15:51:11

   -> Daily DataFrame : 6198 rows, 12 columns

TDMDBPR : Disconnected


In [41]:
''' METRIC by Day '''

v_tm_key_day = 20250101
print(f'\nParameter input...\n')
print(f'   -> v_tm_key_day: {v_tm_key_day}')

src_sale_day_df = chk_src_df.copy()
src_sale_day_df = src_sale_day_df.loc[src_sale_day_df['TM_KEY_DAY']>=v_tm_key_day]
src_sale_day_df = src_sale_day_df.fillna(0).sort_values(by=['METRIC_CD', 'TM_KEY_DAY']).reset_index(drop=True)

mod_col_list = src_sale_day_df.iloc[:, 5:12].columns.tolist()
for col in mod_col_list:
    src_sale_day_df[col] = src_sale_day_df[col].apply(lambda x: format(x, ',.0f'))

print(f'\nCreate : src_sale_day_df...')
print(f'\n   -> DataFrame : {chk_src_df.shape[0]} rows, {chk_src_df.shape[1]} columns')
# print(f'\n{src_sale_day_df.to_string(max_cols=12)}')


Parameter input...

   -> v_tm_key_day: 20250101

Create : src_sale_day_df...

   -> DataFrame : 6198 rows, 12 columns


In [42]:
''' xB1S000101CS : Prepaid Gross Adds : xxx '''

v_tm_key_day = 20250301
v_metric_cd = ['DB1S000101CS', 'TB1S000101CS']
print(f'\nDB1S000101CS : Prepaid Gross Adds : DTAC\nTB1S000101CS : Prepaid Gross Adds : TMH\n')
print(f'   -> v_metric_cd: {v_metric_cd}')
print(f'   -> v_tm_key_day: {v_tm_key_day}')

xB1S000101CS_day_df = chk_src_df.copy()
xB1S000101CS_day_df = xB1S000101CS_day_df.loc[xB1S000101CS_day_df['TM_KEY_DAY']>=v_tm_key_day]
xB1S000101CS_day_df = xB1S000101CS_day_df.loc[xB1S000101CS_day_df['METRIC_CD'].isin(v_metric_cd)]
xB1S000101CS_day_df = xB1S000101CS_day_df.fillna(0).sort_values(by=['PRODUCT', 'METRIC_CD', 'TM_KEY_DAY']).reset_index(drop=True)

mod_col_list = xB1S000101CS_day_df.iloc[:, 5:12].columns.tolist()
for col in mod_col_list:
    xB1S000101CS_day_df[col] = xB1S000101CS_day_df[col].apply(lambda x: format(x, ',.0f'))
    
xB1S000101CS_day_df


DB1S000101CS : Prepaid Gross Adds : DTAC
TB1S000101CS : Prepaid Gross Adds : TMH

   -> v_metric_cd: ['DB1S000101CS', 'TB1S000101CS']
   -> v_tm_key_day: 20250301


,PRODUCT,METRIC_CD,METRIC_NAME,TM_KEY_DAY,LOAD_DATE,C,P,G,H,HH,CCAA,CCAATT
0,PREPAID,DB1S000101CS,Prepaid Gross Adds : DTAC,20250301,2025-04-03 14:03:28,0,"39,049","33,586","33,586","36,734","37,288","37,073"
1,PREPAID,DB1S000101CS,Prepaid Gross Adds : DTAC,20250302,2025-04-03 14:03:28,0,"41,058","33,973","33,973","37,310","37,911","37,496"
2,PREPAID,DB1S000101CS,Prepaid Gross Adds : DTAC,20250303,2025-04-03 14:03:28,0,"35,155","30,340","30,340","33,574","34,442","34,283"
3,PREPAID,DB1S000101CS,Prepaid Gross Adds : DTAC,20250304,2025-04-03 14:03:28,0,"33,235","28,891","28,891","32,039","32,829","32,692"
4,PREPAID,DB1S000101CS,Prepaid Gross Adds : DTAC,20250305,2025-04-03 14:03:28,0,"34,208","29,931","29,931","33,070","33,765","33,632"
...,...,...,...,...,...,...,...,...,...,...,...,...
57,PREPAID,TB1S000101CS,Prepaid Gross Adds : TMH,20250327,2025-04-03 14:03:28,0,"31,786","23,540","23,540","22,696","23,540","23,537"
58,PREPAID,TB1S000101CS,Prepaid Gross Adds : TMH,20250328,2025-04-03 14:03:28,0,"31,270","23,912","23,912","23,173","23,912","23,911"
59,PREPAID,TB1S000101CS,Prepaid Gross Adds : TMH,20250329,2025-04-03 14:03:28,0,"30,075","22,705","22,705","22,080","22,705","22,704"
60,PREPAID,TB1S000101CS,Prepaid Gross Adds : TMH,20250330,2025-04-03 14:03:28,0,"28,425","22,229","22,229","21,569","22,229","22,226"


In [43]:
''' xB2S010100CS : Postpaid Gross Adds B2C : xxx '''

v_tm_key_day = 20250301
v_metric_cd = ['DB2S010100CS', 'TB2S010100CS']
print(f'\nDB2S010100CS : Postpaid Gross Adds : DTAC\nTB2S010100CS : Postpaid Gross Adds : TMH\n')
print(f'   -> v_metric_cd: {v_metric_cd}')
print(f'   -> v_tm_key_day: {v_tm_key_day}')

xB2S010100CS_day_df = chk_src_df.copy()
xB2S010100CS_day_df = xB2S010100CS_day_df.loc[xB2S010100CS_day_df['TM_KEY_DAY']>=v_tm_key_day]
xB2S010100CS_day_df = xB2S010100CS_day_df.loc[xB2S010100CS_day_df['METRIC_CD'].isin(v_metric_cd)]
xB2S010100CS_day_df = xB2S010100CS_day_df.fillna(0).sort_values(by=['PRODUCT', 'METRIC_CD', 'TM_KEY_DAY']).reset_index(drop=True)

mod_col_list = xB2S010100CS_day_df.iloc[:, 5:12].columns.tolist()
for col in mod_col_list:
    xB2S010100CS_day_df[col] = xB2S010100CS_day_df[col].apply(lambda x: format(x, ',.0f'))
    
xB2S010100CS_day_df


DB2S010100CS : Postpaid Gross Adds : DTAC
TB2S010100CS : Postpaid Gross Adds : TMH

   -> v_metric_cd: ['DB2S010100CS', 'TB2S010100CS']
   -> v_tm_key_day: 20250301


,PRODUCT,METRIC_CD,METRIC_NAME,TM_KEY_DAY,LOAD_DATE,C,P,G,H,HH,CCAA,CCAATT
0,POSTPAID,DB2S010100CS,Postpaid Gross Adds B2C : DTAC,20250301,2025-04-03 14:15:44,0,"1,143","1,138","1,138","1,138","1,138","1,138"
1,POSTPAID,DB2S010100CS,Postpaid Gross Adds B2C : DTAC,20250302,2025-04-03 14:15:44,0,888,887,887,887,887,887
2,POSTPAID,DB2S010100CS,Postpaid Gross Adds B2C : DTAC,20250303,2025-04-03 14:15:44,0,"1,362","1,354","1,354","1,354","1,354","1,354"
3,POSTPAID,DB2S010100CS,Postpaid Gross Adds B2C : DTAC,20250304,2025-04-03 14:15:44,0,"1,696","1,684","1,684","1,684","1,684","1,684"
4,POSTPAID,DB2S010100CS,Postpaid Gross Adds B2C : DTAC,20250305,2025-04-03 14:15:44,0,"1,383","1,373","1,373","1,373","1,373","1,373"
...,...,...,...,...,...,...,...,...,...,...,...,...
60,POSTPAID,TB2S010100CS,Postpaid Gross Adds B2C : TMH,20250329,2025-04-03 14:15:44,0,"4,772","3,517","3,517","3,517","3,517","3,517"
61,POSTPAID,TB2S010100CS,Postpaid Gross Adds B2C : TMH,20250330,2025-04-03 14:15:44,0,"5,000","3,688","3,688","3,688","3,688","3,688"
62,POSTPAID,TB2S010100CS,Postpaid Gross Adds B2C : TMH,20250331,2025-04-03 14:15:44,0,"4,391","3,460","3,460","3,460","3,460","3,460"
63,POSTPAID,TB2S010100CS,Postpaid Gross Adds B2C : TMH,20250401,2025-04-03 14:15:44,0,"5,901","4,492","4,492","4,492","4,492","4,492"


In [44]:
''' TB3S000102CS : TOL Gross Adds Connected : Consumer '''

v_tm_key_day = 20250301
v_metric_cd = 'TB3S000102CS'
print(f'\nTB3S000102CS : TOL Gross Adds Connected : Consumer\n')
print(f'   -> v_metric_cd: {v_metric_cd}')
print(f'   -> v_tm_key_day: {v_tm_key_day}')

TB3S000102CS_day_df = chk_src_df.copy()
TB3S000102CS_day_df = TB3S000102CS_day_df.loc[TB3S000102CS_day_df['TM_KEY_DAY']>=v_tm_key_day]
TB3S000102CS_day_df = TB3S000102CS_day_df.loc[TB3S000102CS_day_df['METRIC_CD']==v_metric_cd]
TB3S000102CS_day_df = TB3S000102CS_day_df.fillna(0).sort_values(by=['PRODUCT', 'METRIC_CD', 'TM_KEY_DAY']).reset_index(drop=True)

mod_col_list = TB3S000102CS_day_df.iloc[:, 5:12].columns.tolist()
for col in mod_col_list:
    TB3S000102CS_day_df[col] = TB3S000102CS_day_df[col].apply(lambda x: format(x, ',.0f'))
    
TB3S000102CS_day_df


TB3S000102CS : TOL Gross Adds Connected : Consumer

   -> v_metric_cd: TB3S000102CS
   -> v_tm_key_day: 20250301


,PRODUCT,METRIC_CD,METRIC_NAME,TM_KEY_DAY,LOAD_DATE,C,P,G,H,HH,CCAA,CCAATT
0,TOL,TB3S000102CS,TOL Gross Adds Connected : Consumer,20250301,2025-04-03 14:02:38,0,"1,759","1,376","1,376","1,365","1,376","1,376"
1,TOL,TB3S000102CS,TOL Gross Adds Connected : Consumer,20250302,2025-04-03 14:02:38,0,"1,603","1,211","1,211","1,193","1,211","1,211"
2,TOL,TB3S000102CS,TOL Gross Adds Connected : Consumer,20250303,2025-04-03 14:02:38,0,"1,737","1,392","1,392","1,373","1,392","1,392"
3,TOL,TB3S000102CS,TOL Gross Adds Connected : Consumer,20250304,2025-04-03 14:02:38,0,"1,789","1,449","1,449","1,441","1,449","1,449"
4,TOL,TB3S000102CS,TOL Gross Adds Connected : Consumer,20250305,2025-04-03 14:02:38,0,"1,733","1,417","1,417","1,402","1,417","1,417"
5,TOL,TB3S000102CS,TOL Gross Adds Connected : Consumer,20250306,2025-04-03 14:02:38,0,"1,726","1,408","1,408","1,400","1,408","1,408"
6,TOL,TB3S000102CS,TOL Gross Adds Connected : Consumer,20250307,2025-04-03 14:02:38,0,"1,657","1,348","1,348","1,338","1,348","1,348"
7,TOL,TB3S000102CS,TOL Gross Adds Connected : Consumer,20250308,2025-04-03 14:02:38,0,"1,806","1,334","1,334","1,323","1,334","1,334"
8,TOL,TB3S000102CS,TOL Gross Adds Connected : Consumer,20250309,2025-04-03 14:02:38,0,"1,711","1,265","1,265","1,259","1,265","1,265"
9,TOL,TB3S000102CS,TOL Gross Adds Connected : Consumer,20250310,2025-04-03 14:02:38,0,"1,826","1,418","1,418","1,410","1,418","1,418"


In [45]:
''' TB4S001400CS : TVS Now Gross Adds '''

v_tm_key_day = 20250301
v_metric_cd = 'TB4S001400CS'
print(f'\nTB4S001400CS : TVS Now Gross Adds\n')
print(f'   -> v_metric_cd: {v_metric_cd}')
print(f'   -> v_tm_key_day: {v_tm_key_day}')

TB4S001400CS_day_df = chk_src_df.copy()
TB4S001400CS_day_df = TB4S001400CS_day_df.loc[TB4S001400CS_day_df['TM_KEY_DAY']>=v_tm_key_day]
TB4S001400CS_day_df = TB4S001400CS_day_df.loc[TB4S001400CS_day_df['METRIC_CD']==v_metric_cd]
TB4S001400CS_day_df = TB4S001400CS_day_df.fillna(0).sort_values(by=['PRODUCT', 'METRIC_CD', 'TM_KEY_DAY']).reset_index(drop=True)

mod_col_list = TB4S001400CS_day_df.iloc[:, 5:12].columns.tolist()
for col in mod_col_list:
    TB4S001400CS_day_df[col] = TB4S001400CS_day_df[col].apply(lambda x: format(x, ',.0f'))
    
TB4S001400CS_day_df


TB4S001400CS : TVS Now Gross Adds

   -> v_metric_cd: TB4S001400CS
   -> v_tm_key_day: 20250301


,PRODUCT,METRIC_CD,METRIC_NAME,TM_KEY_DAY,LOAD_DATE,C,P,G,H,HH,CCAA,CCAATT
0,TVS,TB4S001400CS,TVS Now Gross Adds,20250301,2025-04-03 14:18:45,0,24,24,24,24,24,24
1,TVS,TB4S001400CS,TVS Now Gross Adds,20250302,2025-04-03 14:18:45,0,47,47,47,47,47,47
2,TVS,TB4S001400CS,TVS Now Gross Adds,20250303,2025-04-03 14:18:45,0,22,22,22,22,22,22
3,TVS,TB4S001400CS,TVS Now Gross Adds,20250304,2025-04-03 14:18:45,0,23,23,23,23,23,23
4,TVS,TB4S001400CS,TVS Now Gross Adds,20250305,2025-04-03 14:18:45,0,79,79,79,79,79,79
5,TVS,TB4S001400CS,TVS Now Gross Adds,20250306,2025-04-03 14:18:45,0,215,215,215,215,215,215
6,TVS,TB4S001400CS,TVS Now Gross Adds,20250307,2025-04-03 14:18:45,0,162,162,162,162,162,162
7,TVS,TB4S001400CS,TVS Now Gross Adds,20250308,2025-04-03 14:18:45,0,143,143,143,143,143,143
8,TVS,TB4S001400CS,TVS Now Gross Adds,20250309,2025-04-03 14:18:45,0,145,145,145,145,145,145
9,TVS,TB4S001400CS,TVS Now Gross Adds,20250310,2025-04-03 14:18:45,0,87,87,87,87,87,87


## Actual : Issue
->  DIM_CORP_KPI

In [46]:
# ''' Execute Issue by case '''


# # Input parameter
# curr_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
# # v_start_date = 20250101
# print(f'\nParameter input...\n')
# # print(f'   -> v_start_date: {v_start_date}')


# # Connect : TDMDBPR
# src_dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
# src_conn = oracledb.connect(src_dsn)
# print(f'\n{TDMDBPR_db} : Connected')
# src_cur = src_conn.cursor()
# query = (f"""
#     SELECT /*+PARALLEL(8)*/ T.TM_KEY_DAY AS TM_KEY_MTH
#         , T.METRIC_CD, T.METRIC_NAME, T.AREA_TYPE, T.AREA_CD, T.AREA_DESC
#         , T.METRIC_VALUE AS TARGET
#         , A.ACTUAL
#         , O.ORGID_P, O.ZONE_TYPE, O.ORGID_G, O.TDS_SGMD, O.ORGID_H, O.HOP_HINT, O.ORGID_HH, O.D_CLUSTER
        
#     FROM CDSAPPO.DIM_CORP_TARGET T

#     LEFT JOIN (
#         SELECT SUBSTR(TM_KEY_DAY,1,6) AS TM_KEY_MTH
#             , PRODUCT, METRIC_CD, METRIC_NAME, AREA_TYPE, AREA_CD, AREA_DESC
#             , SUM(METRIC_VALUE) AS ACTUAL, MAX(LOAD_DATE) LOAD_DATE
#         FROM CDSAPPO.DIM_CORP_KPI
#         WHERE METRIC_CD = 'TB3S000102CS' --TOL Gross Adds Connected : Consumer
#         AND AREA_TYPE = 'CCAA'
#         AND TM_KEY_DAY >= 20250101
#         GROUP BY SUBSTR(TM_KEY_DAY,1,6), PRODUCT, METRIC_CD, METRIC_NAME, AREA_TYPE, AREA_CD, AREA_DESC
#     ) A
#         ON A.METRIC_CD = T.METRIC_CD
#         AND A.AREA_CD = T.AREA_CD
#         AND A.TM_KEY_MTH = T.TM_KEY_DAY
        
#     LEFT JOIN (
#         SELECT DISTINCT ORGID_P, ZONE_TYPE, ORGID_G, TDS_SGMD, ORGID_H, HOP_HINT, ORGID_HH, D_CLUSTER
#             , SUBSTR(CCAATT,1,2) AS PROVINCE_CD, PROVINCE_ENG 
#             , SUBSTR(CCAATT,1,4) AS CCAA, DISTRICT_EN
#         FROM CDSAPPO.DIM_MOOC_AREA NOLOCK
#         WHERE TEAM_CODE <> 'ไม่ระบุ' AND REMARK <> 'Dummy'
#     ) O
#         ON O.CCAA = T.AREA_CD

#     WHERE T.METRIC_CD = 'TB3S000102CS'
#     AND T.AREA_TYPE = 'CCAA'
#     AND T.TM_KEY_DAY >= 202501

#     --ORDER BY T.TM_KEY_DAY, T.METRIC_CD, T.AREA_TYPE, O.ZONE_TYPE, O.ORGID_G, O.ORGID_H, O.ORGID_HH
# """)


# try:
#     execute_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
#     print(f'\n   -> Execute query... {execute_datetime}')

#     # Get : Summary DataFrame
#     src_cur.execute(query)
#     rows = src_cur.fetchall()
#     chk_src_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in src_cur.description])
#     print(f'\n   -> Issue DataFrame : {chk_src_df.shape[0]} rows, {chk_src_df.shape[1]} columns')

#     src_cur.close()


# except oracledb.DatabaseError as e:
#     print(f'\nError with Oracle : {e}')


# finally:
#     src_conn.close()
#     print(f'\n{TDMDBPR_db} : Disconnected')

In [47]:
# ''' TB3S000102CS : TOL Gross Adds Connected : Consumer '''

# v_tm_key_mth = 202501
# # v_metric_cd = 'TB3S000102CS'
# print(f'\nTB3S000102CS : TOL Gross Adds Connected : Consumer\n')
# # print(f'   -> v_metric_cd: {v_metric_cd}')
# print(f'   -> v_tm_key_mth: {v_tm_key_mth}')

# TB3S000102CS_issue_df = chk_src_df.copy()
# TB3S000102CS_issue_df = TB3S000102CS_issue_df.loc[TB3S000102CS_issue_df['TM_KEY_MTH']==v_tm_key_mth]
# TB3S000102CS_issue_df = TB3S000102CS_issue_df.loc[TB3S000102CS_issue_df['ACTUAL'] > 0]
# # TB3S000102CS_issue_df = TB3S000102CS_issue_df.loc[TB3S000102CS_issue_df['ACTUAL'].isna()]
# # TB3S000102CS_issue_df = TB3S000102CS_issue_df.loc[TB3S000102CS_issue_df['METRIC_CD']==v_metric_cd]
# TB3S000102CS_issue_df = TB3S000102CS_issue_df.fillna(0).sort_values(by=['TM_KEY_MTH', 'METRIC_CD']).reset_index(drop=True)

# mod_col_list = TB3S000102CS_issue_df.iloc[:, 6:8].columns.tolist()
# for col in mod_col_list:
#     TB3S000102CS_issue_df[col] = TB3S000102CS_issue_df[col].apply(lambda x: format(x, ',.0f'))
    
# # TB3S000102CS_issue_df
# print(f'\n{TB3S000102CS_issue_df.to_string(max_cols=100)}')

## Output File
->  DIM_CORP_KPI

In [48]:
# ''' Execute Sales Monthly '''


# # Input parameter
# curr_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
# v_start_date = 20240101
# print(f'\nParameter input...\n')
# print(f'   -> v_start_date: {v_start_date}')


# # Connect : TDMDBPR
# src_dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
# src_conn = oracledb.connect(src_dsn)
# print(f'\n{TDMDBPR_db} : Connected')
# src_cur = src_conn.cursor()
# query = (f"""
#     SELECT /*+PARALLEL(8)*/ 
#         SUBSTR(TM_KEY_DAY,1,4) AS TM_KEY_YR, SUBSTR(TM_KEY_DAY,1,6) AS TM_KEY_MTH
#         , CASE WHEN REGEXP_LIKE(METRIC_CD, 'CS$|CG$') THEN 'Y2025' ELSE 'Y2024' END VER_FLAG
#         , PRODUCT, COMP_CD, METRIC_CD, METRIC_NAME, REMARK, UPD_TM, PPN_TM
#         , SUM(CASE WHEN AREA_TYPE = 'P' THEN METRIC_VALUE END) P_ACTUAL
#         , SUM(CASE WHEN AREA_TYPE = 'G' THEN METRIC_VALUE END) G
#         , SUM(CASE WHEN AREA_TYPE = 'H' THEN METRIC_VALUE END) H
#         , SUM(CASE WHEN AREA_TYPE = 'HH' THEN METRIC_VALUE END) HH
#         , SUM(CASE WHEN AREA_TYPE = 'CCAA' THEN METRIC_VALUE END) CCAA
#         , SUM(CASE WHEN AREA_TYPE = 'CCAATT' THEN METRIC_VALUE END) CCAATT
#         , SUM(CASE WHEN AREA_CD = '016' THEN METRIC_VALUE END) "016"
#         , SUM(CASE WHEN AREA_CD = '040' THEN METRIC_VALUE END) "040"
#         , SUM(CASE WHEN AREA_CD = '080' THEN METRIC_VALUE END) "080"
#         , MIN(TM_KEY_DAY) AS START_DAY, MAX(TM_KEY_DAY) AS END_DAY, MAX(LOAD_DATE) LOAD_DATE
#     FROM CDSAPPO.DIM_CORP_KPI
#     WHERE METRIC_CD NOT LIKE 'Error%'
#     AND EXISTS (
#         SELECT 1 -->> AREA_CD IN (P, G, H, R, HH, CCAA, CCAATT)
#         FROM (
#             SELECT DISTINCT ORGID_P AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
#             UNION SELECT DISTINCT ORGID_G AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
#             UNION SELECT DISTINCT ORGID_R AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
#             UNION SELECT DISTINCT ORGID_H AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
#             UNION SELECT DISTINCT ORGID_HH AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
#             UNION SELECT DISTINCT SUBSTR(CCAATT,1,4) AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
#             UNION SELECT DISTINCT CCAATT AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
#         ) TMP
#         WHERE TMP_CD = AREA_CD )
#     AND TM_KEY_DAY >= {v_start_date}
#     GROUP BY SUBSTR(TM_KEY_DAY,1,4), SUBSTR(TM_KEY_DAY,1,6), PRODUCT, COMP_CD, METRIC_CD, METRIC_NAME, REMARK, UPD_TM, PPN_TM
#     --ORDER BY 2, 3, PRODUCT, COMP_CD, METRIC_CD 
# """)


# try:
#     execute_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
#     print(f'\n   -> Execute query... {execute_datetime}')

#     # Get : Sales Monthly DataFrame
#     src_cur.execute(query)
#     rows = src_cur.fetchall()
#     monthly_sales_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in src_cur.description])
#     print(f'\n   -> Sales Monthly DataFrame : {monthly_sales_df.shape[0]} rows, {monthly_sales_df.shape[1]} columns')

#     src_cur.close()


# except oracledb.DatabaseError as e:
#     print(f'\nError with Oracle : {e}')


# finally:
#     src_conn.close()
#     print(f'\n{TDMDBPR_db} : Disconnected')

In [49]:
# ''' Generate CSV file '''

# op_dir = 'data'
# op_monthly_file = f'sales_monthly_{str_curr_dt}'

# monthly_sales_df.to_csv(f'{op_dir}/tmp_{op_monthly_file}.csv', index=False, encoding='utf-8')
# print(f'\n   -> Generate "tmp_{op_monthly_file}.csv" successfully')

# monthly_sales_df.tail(3)